In [0]:
catalog_name = "dmc_tallerfinaldatabricks_mariovento"

schema_silver = "silver"
schema_gold = "gold"

In [0]:
from pyspark.sql.functions import current_timestamp, col

In [0]:
productos_silver = spark.table(f"{catalog_name}.{schema_silver}.productos")
 
# Crea el DataFrame 'dim_producto' agregando columnas técnicas y seleccionando campos relevantes
dim_producto = (
    productos_silver
    .withColumn("sk_producto", col("id_producto")) # Agrega una clave surrogate única para cada producto
    .withColumn("creation_date", current_timestamp()) # Agrega la fecha y hora de creación del registro
    .select(
        col("sk_producto"), # Clave surrogate del producto
        col("id_producto"), # ID original del producto
        col("nombre"), # Nombre del producto
        col("categoria"), # Categoría del producto
        col("precio"), # Precio del producto
        col("creation_date") # Fecha de creación del registro
    )
    .orderBy(col("id_producto").asc())
)

In [0]:
dim_producto.write.format("delta").mode("overwrite").saveAsTable(f"{catalog_name}.{schema_gold}.dim_producto")

In [0]:
%sql
select * from dmc_tallerfinaldatabricks_mariovento.gold.dim_producto